# BERT Training

This Jupyter notebook contains the training of a BERT model and the necessary steps to convert the data to be usable for the training.

Author: Chrisitan Bauer

## Sources:

- https://towardsdatascience.com/how-to-train-bert-for-q-a-in-any-language-63b62c780014
- https://huggingface.co/transformers/v3.4.0/model_doc/bert.html#bertforquestionanswering
- https://huggingface.co/docs/datasets/metrics.html


## WandB Montitor Output

https://wandb.ai/my-god-its-full-of-stars/BERT%20Training?workspace=user-my-god-its-full-of-stars

In [1]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from transformers import (
    BertForQuestionAnswering, BertTokenizerFast, 
    AdamW, 
    DistilBertForQuestionAnswering, DistilBertTokenizer)

# fix BERT to DistillBERT transition
# https://fixexception.com/transformers/char-to-token-is-not-available-when-using-python-based-tokenizers/
# from transformers.tokenization_utils_base import BatchEncoding
# from tokenizers import Encoding as EncodingFast
    
from datasets import load_dataset, load_metric
import wandb
from tqdm.notebook import tqdm, trange, tqdm_notebook # show progress bar
from sklearn.metrics import f1_score

## Load Dataset and Split into Training and Validation Datasets

In [2]:
dataset = load_dataset("squad")
train_dataset, validation_dataset = dataset['train'], dataset['validation']

# metric = load_metric('bertscore')
# train_dataset

Reusing dataset squad (/home/christian/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00, 472.78it/s]


### Question-Answer Set Structure

Each Set conists of the following compontens:

- **Question**: string contianing the question Bert gets asked.
- **Context**: a larger sequence containing the answer to the question.
- **Answer**: a part of the context, that answers the question.

Given the question and context, the model must be able to read both of them and return the token positions of the predicted answer within the context.

In [3]:
train_dataset[0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [4]:
# answer_start = train_dataset[0]['answers']['answer_start'][0]
# answer_start

In [5]:
# answer_length = len(train_dataset[0]['answers']['text'][0])
# answer_length

In [6]:
# train_dataset[0]['context'][answer_start:answer_start+answer_length]

## Formatting QA-Sets

Before the training can start, the answer section of the sets needs to be reformatted.
The key `answer_end` gets added to the dictionary and for easier accessing, the lists containing only one element get removed by the element itself.

In [7]:
def add_end_idx(answers, contexts):
    new_answers = []
    # loop through each answer-context pair
    for answer, context in zip(answers, contexts):
        # quick reformating to remove lists
        answer['text'] = answer['text'][0]
        answer['answer_start'] = answer['answer_start'][0]
        # gold_text refers to the answer we are expecting to find in context
        gold_text = answer['text']
        # we already know the start index
        start_idx = answer['answer_start']
        # and ideally this would be the end index...
        end_idx = start_idx + len(gold_text)

        # ...however, sometimes squad answers are off by a character or two
        if context[start_idx:end_idx] == gold_text:
            # if the answer is not off :)
            answer['answer_end'] = end_idx
        else:
            # this means the answer is off by 1-2 tokens
            for n in [1, 2]:
                if context[start_idx-n:end_idx-n] == gold_text:
                    answer['answer_start'] = start_idx - n
                    answer['answer_end'] = end_idx - n
        new_answers.append(answer)
    return new_answers

def prep_data(dataset):
    questions = dataset['question']
    contexts = dataset['context']
    answers = add_end_idx(
        dataset['answers'],
        contexts
    )
    return {
        'question': questions,
        'context': contexts,
        'answers': answers
    }

## Initialize Model, Tokenizer and Optimizer

In [8]:
train_dataset = prep_data(train_dataset)
validation_dataset = prep_data(validation_dataset)
train_dataset['answers'][:3]

[{'text': 'Saint Bernadette Soubirous',
  'answer_start': 515,
  'answer_end': 541},
 {'text': 'a copper statue of Christ', 'answer_start': 188, 'answer_end': 213},
 {'text': 'the Main Building', 'answer_start': 279, 'answer_end': 296}]

### Tokenization

For Bert to be able to read the SQUAD dataset, tokenization is required.
For `context` and `question` the standard `tokenizer()` method can be used.

This method encodes both `context` and `question` strings into single arrays of tokens.
This will act as the input for the QA training.

In [9]:
pretrained_model_name: str = 'bert-large-uncased-whole-word-masking-finetuned-squad'
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
# model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-cased-distilled-squad')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased-distilled-squad')

# train = tokenizer(train_dataset['context'], train_dataset['question'],
#                           truncation=True, padding='max_length',
#                           max_length=512, return_tensors='pt')

# validation = tokenizer(validation_dataset['context'], validation_dataset['question'],
#                           truncation=True, padding='max_length',
#                           max_length=512, return_tensors='pt')
            

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [10]:
# tokenizer.decode(train['input_ids'][0])[:855]
# tokenizer.decode(validation['input_ids'][0])[:855]


In [11]:
token = tokenizer('Computer Vision research has made a giant leap since AlexNet')
token

{'input_ids': [101, 3274, 4432, 2470, 2038, 2081, 1037, 5016, 11679, 2144, 4074, 7159, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [12]:
tokenizer.decode(token['input_ids'])

'[CLS] computer vision research has made a giant leap since alexnet [SEP]'

### Tokenization Part 2

Since tokens get fed into Bert, the start and end positions of the tokens need to be provided.

This is done by the method `add_token_positions()`.

In [13]:
def add_token_positions(encodings, answers):
    # initialize lists to contain the token indices of answer start/end
    start_positions = []
    end_positions = []
    # for i in tqdm(range(len(answers))):
    for i in range(len(answers)):
        # append start/end token position using char_to_token method
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        # end position cannot be found, char_to_token found space, so shift position until found
        shift = 1
        while end_positions[-1] is None:
            # encodings = BatchEncoding(answers[i]['answers_end'] -shift, encoding=EncodingFast())
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end'] - shift)
            shift += 1
    # update our encodings object with the new token-based start/end positions
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})


In [14]:
# apply function to the data
# add_token_positions(train, train_dataset['answers'])
# add_token_positions(validation, validation_dataset['answers'])

### This function adds two more tensors to the `Encoding` object (which will be fed into Bert)

- the `start_positions` and
- the `end_positions` tensors.


In [15]:
# train.keys()

In [16]:
# train['start_positions'][:5], train['end_positions'][:5]

### Training

For the training, Pytorch is used and the dataset will be converted to a Pytorch Dataset.

In [17]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

# train_dataset = SquadDataset(train)
# validation_dataset = SquadDataset(validation)

In [18]:
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# len(train_loader)

In [19]:
# validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)
# len(validation_loader)

In [20]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='min', 
    threshold=0.001, 
    cooldown=0, 
    eps=1e-07, 
    patience=2, 
    verbose=True, 
    min_lr=1e-5, 
    factor=0.5)

### Helper Function for partially loading the dataset

This is required because otherwise an error occurs because to much RAM is allocated.

In [21]:
def get_next_dataloader(start_index: int, end_index: int, shuffle: bool = True, dataset_key: str = 'train') -> DataLoader:
    part_dataset = dataset[dataset_key][start_index:end_index]
    part_dataset = prep_data(part_dataset)
    part_train = tokenizer(part_dataset['context'], part_dataset['question'],
                          truncation=True, padding='max_length',
                          max_length=512, return_tensors='pt')
    tokenizer.decode(part_train['input_ids'][0])[:855]
    # apply function to the data
    add_token_positions(part_train, part_dataset['answers'])
    part_dataset = SquadDataset(part_train)
    return DataLoader(part_dataset, batch_size=32, shuffle=shuffle)


### Initializing WANDB for Monitoring

In [22]:
wandb.init(
    project='BERT Training',
    config={
        'batch_size': 32,
        'dataset': 'SQUAD 2.0'
    })

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: my-god-its-full-of-stars (use `wandb login --relogin` to force relogin)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [23]:
def convert_all_input_ids_to_tokens(input_ids):
    return [tokenizer.convert_ids_to_tokens(ids) for ids in input_ids]

#### Initialize Training Preliminaries

In [24]:
max_train_batch_length: int = 2738
max_validation_batch_length: int = 331

num_episodes: int = 100
save_zones = [epoch for epoch in range(10, 101, 25)]
step = 5

training_losses = []
validation_losses = []

In [25]:

def train_model(start_epoch: int, end_epoch: int, model, optimizer, scheduler):
    
    # for epoch in trange(start_epoch, end_epoch):
    for epoch in trange(start_epoch, end_epoch):
        print(f'Epoch: {epoch}')

        model.train()

        train_accuracy_start = 0
        train_accuracy_end = 0
        train_total = 0

        train_accuracy_start_ = []
        train_accuracy_end_ = []
        train_f1_start = []
        train_f1_end = []
        f1_start = 0
        f1_end = 0
    # roberta-small
        # for start_range in trange(0, max_train_batch_length, step):
        for start_range in range(0, max_train_batch_length, step):
            end_range = min(max_train_batch_length, start_range + step)

            # loop = tqdm_notebook(get_next_dataloader(start_range, end_range), desc=f'<{start_range}:{end_range}>')
            # loop.set_description(f'Epoch {epoch + 1} Training <{start_range}:{end_range}>')
            loop = get_next_dataloader(start_range, end_range)


            for batch in loop:
                optimizer.zero_grad()

                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                start_positions = batch['start_positions'].to(device)
                end_positions = batch['end_positions'].to(device)

                outputs = model(input_ids, attention_mask=attention_mask,
                                start_positions=start_positions,
                                end_positions=end_positions)
                
    
                start_scores = outputs.start_logits
                end_scores = outputs.end_logits
                answer_start = torch.argmax(start_scores, dim=1)
                answer_end = torch.argmax(end_scores, dim=1)

                train_accuracy_start = (start_positions == answer_start).sum().float()
                train_accuracy_end = (end_positions == answer_end).sum().float()
                train_accuracy_start_.append(train_accuracy_start)
                train_accuracy_end_.append(train_accuracy_end)

                # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html
                f1_start = f1_score(start_positions.cpu(), answer_start.cpu(), average='weighted', zero_division=0)
                f1_end = f1_score(end_positions.cpu(), answer_end.cpu(), average='weighted', zero_division=0)
                train_f1_start.append(f1_start)
                train_f1_end.append(f1_end)

                train_total += len(start_positions)
            
                loss = outputs.loss
                loss.backward()
                optimizer.step()
                scheduler.step(loss)

                # loop.set_postfix(loss=loss.item())

        training_losses.append(loss.item())
        train_accuracy_start = sum(train_accuracy_start_) / len(train_accuracy_start_)
        train_accuracy_end = sum(train_accuracy_end_) / len(train_accuracy_end_)
        f1_start = sum(train_f1_start) / len(train_f1_start)
        f1_end = sum(train_f1_end) / len(train_f1_end)

        wandb.log({
            'training loss': loss.item(),
            'training loss average': sum(training_losses) / len(training_losses),
            'training accuracy average': (train_accuracy_start + train_accuracy_end) / 2,
            'training accuracy start position': train_accuracy_start,
            'training accuracy end position': train_accuracy_end,
            'training f1 score average': (f1_start + f1_end) / 2,
            'training f1 score start position': f1_start,
            'training f1 score end position': f1_end 
        })
        

        validation_accuracy_start = 0
        validation_accuracy_end = 0
        validation_total = 0

        validation_accuracy_start_ = []
        validation_accuracy_end_ = []
        validation_f1_start = []
        validation_f1_end = []

        f1_start = 0
        f1_end = 0

        # for start_range in trange(0, max_validation_batch_length, step):
        for start_range in range(0, max_validation_batch_length, step):
            end_range = min(max_validation_batch_length, start_range + step)
            # loop = tqdm(get_next_dataloader(start_range, end_range, shuffle=False, dataset_key='validation'))
            loop = get_next_dataloader(start_range, end_range, shuffle=False, dataset_key='validation')
            # loop.set_description(f'Epoch {epoch + 1} Validation <{start_range}:{end_range}>')

                

            with torch.no_grad():
                model.eval()
                for batch in loop:
                    input_ids = batch['input_ids'].to(device)
                    attention_mask = batch['attention_mask'].to(device)
                    start_positions = batch['start_positions'].to(device)
                    end_positions = batch['end_positions'].to(device)

                    outputs = model(input_ids, attention_mask=attention_mask,
                                    start_positions=start_positions,
                                    end_positions=end_positions)

                    start_scores = outputs.start_logits
                    end_scores = outputs.end_logits
                    answer_start = torch.argmax(start_scores, dim=1)
                    answer_end = torch.argmax(end_scores, dim=1)

                    validation_accuracy_start = (start_positions == answer_start).sum().float()
                    validation_accuracy_end = (end_positions == answer_end).sum().float()
                    validation_accuracy_start_.append(validation_accuracy_start)
                    validation_accuracy_end_.append(validation_accuracy_end)

       
                    f1_start = f1_score(start_positions.cpu(), answer_start.cpu(), average='weighted', zero_division=0)
                    validation_f1_start.append(f1_start)
                    f1_end = f1_score(end_positions.cpu(), answer_end.cpu(), average='weighted', zero_division=0)
                    validation_f1_end.append(f1_end)

                    validation_total += len(start_positions)
                    
                    loss = outputs.loss

                    # loop.set_postfix(loss=loss.item())

                    scheduler.step(loss)

        validation_losses.append(loss.item())
        validation_accuracy_start = sum(validation_accuracy_start_) / len(validation_accuracy_start_)
        validation_accuracy_end = sum(validation_accuracy_end_) / len(validation_accuracy_end_)
        f1_start = sum(validation_f1_start) / len(validation_f1_start)
        f1_end = sum(validation_f1_end) / len(validation_f1_end)
        wandb.log({
            'validation loss': loss.item(), 
            'validation loss average': sum(validation_losses) / len(validation_losses),
            'validation accuracy': (validation_accuracy_start + validation_accuracy_end) / 2,
            'validation accuracy start position': validation_accuracy_start,
            'validation accuracy end position': validation_accuracy_end,
            'validation f1 score average': (f1_start + f1_end) / 2,
            'validation f1 score start position': f1_start,
            'validation f1 score end position': f1_end
        })

    torch.save(model.state_dict(), f'models/bert_model_{epoch}.pt')
    torch.save(optimizer.state_dict(), f'models/optimizer_{epoch}.pt')
    torch.save(scheduler.state_dict(), f'models/scheduler_{epoch}.pt')


In [26]:
def clean_memory():
    del model
    del optimizer
    del scheduler

def init_model(epoch: int = 0):
    model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

    model.to(device)
    optimizer = AdamW(model.parameters(), lr=1e-2)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 
        mode='min',  
        eps=1e-07, 
        verbose=True, 
        min_lr=1e-3, 
        factor=0.5)

    if epoch != -1:
        model_dict = torch.load(f'models/bert_model_{epoch}.pt')
        model.load_state_dict(model_dict)
        optimizer_dict = torch.load(f'models/optimizer_{epoch}.pt')
        optimizer.load_state_dict(optimizer_dict)
        scheduler_dict = torch.load(f'models/scheduler_{epoch}.pt')
        scheduler.load_state_dict(scheduler_dict)

    return model, optimizer, scheduler


In [27]:
        
def get_last_saved_epoch(path: str = 'models/') -> int:
    from os import walk
    files = []
    for (_, _, filenames) in walk(path):
        filenames = [file for file in filenames if file.endswith('.pt')]
        files.extend(filenames)

    epoch_list = [int(file.split('.')[0].split('_').pop()) for file in files]

    return max(epoch_list) if len(epoch_list) > 0 else -1


In [28]:
saved_epoch = get_last_saved_epoch()
model, optimizer, scheduler = init_model(saved_epoch)
optimizer.param_groups[0]['lr'] = 0.1
epochs_to_learn: int = 10
# for start_index in trange(saved_epoch, saved_epoch + 15, epochs_to_learn):
saved_epoch += 1
torch.cuda.empty_cache()
train_model(saved_epoch, saved_epoch + epochs_to_learn, model, optimizer, scheduler)



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 46


/tmp/ipykernel_19820/1725351215.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_19820/1725351215.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_19820/1725351215.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_19820/1725351215.py:6: UserWarning: To copy const

Epoch    21: reducing learning rate of group 0 to 5.0000e-02.


...

/tmp/ipykernel_19820/1725351215.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_19820/1725351215.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_19820/1725351215.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_19820/1725351215.py:6: UserWarning: To copy const

Epoch    76: reducing learning rate of group 0 to 1.5625e-03.


/tmp/ipykernel_19820/1725351215.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_19820/1725351215.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_19820/1725351215.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_19820/1725351215.py:6: UserWarning: To copy const

Epoch    87: reducing learning rate of group 0 to 1.0000e-03.


/tmp/ipykernel_19820/1725351215.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_19820/1725351215.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_19820/1725351215.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_19820/1725351215.py:6: UserWarning: To copy const